In [1]:
import math
import numpy as np
from sklearn.preprocessing import normalize

In [2]:
# def shift_alphabet_char(char, n):
#     if 'a' <= char <= 'z':
#         shifted_char = chr(((ord(char) - ord('a') + n) % 26) + ord('a'))
#         return shifted_char
#     else:
#         return char

# # Read the text from the input file
# with open("/Users/chun/IdeaProjects/CS-271/output.txt", 'r') as infile:
#     input_text = infile.read().replace(' ', '')
# key = 10
# # Apply the shift operation to each character in the text
# shifted_text = ''.join([shift_alphabet_char(char, key) for char in input_text])

# # Write the shifted text to the output file
# with open("/Users/chun/IdeaProjects/CS-271/shifted_output.txt", 'w') as outfile:
#     outfile.write(shifted_text)


shifted = []
obv = []
ori = []
key = 1
with open("/Users/chun/IdeaProjects/CS-271/brown.txt", 'r') as file:
    for line in file:
        for char in line:
            if char.isalpha() and 'a' <= char.lower() <= 'z':
                char = char.lower()
                #print(char.lower())
                ori.append(char.lower())
                shifted.append(chr(((ord(char) - ord('a') + key) % 26) + ord('a')))
                obv.append((ord(char.lower()) - ord('a') + key )%26)
            else:
                continue

ori_str = "".join(ori[0:1000])
with open("ori_str.txt", 'w') as file:
    file.write(ori_str)
with open("shifted.txt", 'w') as file:
    file.write("".join(shifted[0:1000]))
obv = obv[0:1000]
print(obv[0:10])
print(len(obv))

[20, 8, 5, 6, 21, 12, 20, 15, 14, 3]
1000


In [3]:
##init

# N = 2 #number of state
# M = 27
##create obv
# obv = []
# ori = []
# key = 10
# with open("/Users/chun/IdeaProjects/CS-271/brown.txt", 'r') as file:
#     for line in file:
#         for char in line:
#             if char == " ":
#                 continue
#             else:
#                 ori.append(char.lower())
#                 obv.append(chr(((ord(char.lower()) - ord('a') + key) % 26) + ord('a')))

# ori_str = "".join(ori)
# with open("ori_str.txt", 'w') as file:
#     file.write(ori_str)
# print(obv[0:10])
##initlizing a,b,pi
T = len(obv)
M = 26
N = 26
a = np.random.uniform(1/N - 1/(10*N),1/N + 1/(10*N),(N,N))
a = normalize(a, axis=1, norm='l1')
b = np.random.uniform(1/M - 1/(10*M),1/M + 1/(10*M),(N,M))
b = normalize(b, axis=1, norm='l1')
pi = np.random.uniform(1/N - 1/(10*N),1/N + 1/(10*N),(1,N))
pi = normalize(pi , axis=1, norm='l1')[0]

# a = np.matrix([[.7,.3],[.4,.6]])
# b = np.matrix([[.1,.4,.5], [.7,.2,.1]])
# pi = [.6,.4]
# obv = [0,1,0,2]

maxlters = 100
# a = np.zeros((N,N))
# b = np.zeros((N,M))
iters = 0
oldLogProb = -math.inf

In [4]:
import numpy as np

# Initialize a 26x26 matrix to store the frequencies
matrix_size = 26
alphabet_matrix = np.zeros((matrix_size, matrix_size))

# Function to convert a character to its corresponding index (0-25 for 'a'-'z')
def char_to_index(char):
    return ord(char) - ord('a')

# Specify the input file path
input_file = '/Users/chun/IdeaProjects/CS-271/brown.txt'

# Open the input file for reading
with open(input_file, 'r') as file:
    char_count = 0  # Initialize character count
    
    # Read the file in chunks until one million characters are processed
    while char_count < 1000000:
        chunk = file.read(1024)  # Read a chunk of up to 1024 characters
        
        # Iterate through the chunk and count alphabet pairs
        for i in range(len(chunk) - 1):
            char1 = chunk[i]
            char2 = chunk[i + 1]

            # Check if both characters are alphabet characters
            if char1.isalpha() and char2.isalpha():
                char1 = char1.lower()
                char2 = char2.lower()
                index1 = char_to_index(char1)
                index2 = char_to_index(char2)
                alphabet_matrix[index1][index2] += 1

        char_count += len(chunk)  # Update character count

alphabet_matrix += 5
np.savetxt("211d_matrix.csv", alphabet_matrix, delimiter=' ')
a = alphabet_matrix

In [5]:
##Caculating alpha_0:
# alpha = np.zeros((T,N))
# c = np.zeros(T)
# ## i is which state, so needs to go in front
# ## 0 is the number of obversation
# for i in range(0, N):
#     tmp = pi[i] * b[i,obv[0]]
#     alpha[0,i] = tmp
#     c[0] += alpha[0][i]
# c[0] = 1/c[0]
# for i in range(0,N):
#     alpha[0,i] *= c[0]
# print(alpha)

In [6]:
c = np.zeros(T)
alpha = np.zeros((T,N))
def alpha_t(a,b,obv,c,T,N):
#     print("alpha_t")
    ##Compute alpha_t(i)
##t from 1 to 2 to go over the last 2 obv
    ## i is which state, so needs to go in front
    ## 0 is the number of obversation
    for i in range(0, N):
        tmp = pi[i] * b[i,obv[0]]
        alpha[0,i] = tmp
        c[0] += alpha[0][i]
    c[0] = 1/c[0]
    for i in range(0,N):
        alpha[0,i] *= c[0]
    #print(alpha)
    for t in range(1, T):
        c[t] = 0
        for i in range(0, N):
            alpha[t,i] = 0
            ## i is state and t is obv, j is previous state
            for j in range(0,N):
                alpha[t,i] += alpha[t-1,j]*a[j,i]
            alpha[t,i] *= b[i,obv[t]]
            c[t] += alpha [t,i]
        c[t] = 1/ c[t]
        for i in range(0,N):
            alpha[t,i] *= c[t]
    return alpha
alpha = alpha_t(a=a,b=b,c=c,obv=obv,T=T,N=N)

In [7]:
# ##backward init
# beta = np.zeros((T,N))
# for row in beta:
#     row[-1] = 1
# for i in range(0, N):
#     beta[T-1,i] = c[T-1]

In [8]:
##beta-pass
def backward(a,b,c,T,N):
    beta = np.zeros((T,N))
    for row in beta:
        row[-1] = 1
    for i in range(0, N):
        beta[T-1,i] = c[T-1]
#     print("backward")
    for t in range(T-2,-1,-1):
        for i in range(0, N):
            beta[t,i] = 0
            for j in range(0, N):
                beta[t,i] += a[i,j] * b[j,obv[t+1]] * beta[t+1,j]
            beta[t,i] *= c[t]
    return beta
beta = backward(a=a,b=b,c=c,T=T,N=N)
beta

array([[5.80372426e+01, 1.30975094e+01, 2.98892738e+01, ...,
        1.44089727e+00, 3.61992553e+00, 9.11489805e-01],
       [1.49384005e-03, 3.53124289e-04, 7.78059500e-04, ...,
        3.72317307e-05, 9.88792856e-05, 2.45571247e-05],
       [9.48462371e-04, 2.03195943e-04, 4.56634083e-04, ...,
        2.25474369e-05, 5.75205566e-05, 1.42112816e-05],
       ...,
       [8.37797071e-04, 2.39702080e-04, 4.90581438e-04, ...,
        2.18619251e-05, 6.52500469e-05, 1.71053689e-05],
       [1.02464389e-03, 2.04139180e-04, 3.91377930e-04, ...,
        2.28893920e-05, 5.33810755e-05, 1.29741201e-05],
       [6.81737381e-04, 6.81737381e-04, 6.81737381e-04, ...,
        6.81737381e-04, 6.81737381e-04, 6.81737381e-04]])

In [9]:
# 

In [10]:
##Gammas
#a,b,alpha,beta,T,N
#a=a,b=b,alpha=alpha,beta=beta,T=T,N=N
def cal_gammas(a,b,alpha,beta,T,N):
    gammas = np.zeros((T,N))
    de_gammas = np.zeros((T,N,N))
    #print(T)
    for t in range(0, T-1):
        denom = 0
        for i in range(0, N):
            for j in range(0, N):
                denom += alpha[t,i] * a[i,j] * b[j,obv[t+1]] * beta[t+1,j]
        for i in range(0, N):
            gammas[t,i] = 0
            for j in range(0, N):
                de_gammas[t,i,j] = alpha[t,i] * a[i,j] * b[j,obv[t+1]]*beta[t+1,j] / denom
                gammas[t,i] += de_gammas[t,i,j]
    denom = 0
    for i in range(0, N):
        denom += alpha[T-1,i]
    for i in range(0, N):
        gammas[T-1,i] = alpha[T-1,i]/denom
    return gammas,de_gammas
gammas,de_gammas = cal_gammas(a=a,b=b,alpha=alpha,beta=beta,T=T,N=N)
# print(gammas)
print(gammas,de_gammas)

[[8.75435897e-02 1.83576367e-02 4.39253146e-02 ... 2.20487202e-03
  6.01121184e-03 1.31555083e-03]
 [1.06492800e-01 2.35544013e-03 1.72772328e-02 ... 9.02922255e-05
  1.91898383e-03 3.38542352e-05]
 [8.33927487e-02 2.26324129e-03 2.11081757e-02 ... 1.64871285e-04
  1.81663533e-03 3.18631415e-05]
 ...
 [7.60922172e-02 2.30901133e-03 2.46209474e-02 ... 1.41922213e-04
  1.96070930e-03 3.35834552e-05]
 [1.03267406e-01 1.98393498e-03 1.67861478e-02 ... 1.50684723e-04
  1.62776467e-03 2.63159856e-05]
 [6.44389105e-02 6.54756755e-03 3.24845358e-02 ... 4.44077102e-03
  1.89499772e-02 1.22384545e-03]] [[[2.60354822e-05 5.98964034e-04 2.72182494e-03 ... 1.06860310e-05
   3.29548974e-04 4.56050730e-06]
  [2.74691898e-03 4.24742400e-05 1.60770894e-05 ... 2.48237225e-07
   1.54662930e-04 1.85861218e-07]
  [7.19925867e-03 3.07134839e-06 4.19316530e-04 ... 2.60278549e-07
   3.35193172e-05 2.33852217e-07]
  ...
  [4.10583827e-04 2.66500903e-06 1.55775123e-04 ... 3.25214927e-07
   2.26214425e-06 2.0291

In [11]:
print(a)
print("b")
print(b)
print("pi")
print(pi)
print("alpha")
print(alpha)
print("beta")
print(beta)
print("gammas")
print(gammas)
print("de_gammas")
print(de_gammas)

[[1.1000e+01 1.1400e+03 2.5170e+03 2.4940e+03 5.1000e+01 5.3600e+02
  1.3440e+03 1.0000e+02 2.2330e+03 7.2000e+01 6.8600e+02 6.2100e+03
  1.9620e+03 1.1997e+04 9.1000e+01 9.9200e+02 9.0000e+00 7.1660e+03
  5.8320e+03 8.7290e+03 7.9200e+02 1.1980e+03 4.0600e+02 2.0000e+02
  2.0690e+03 1.1400e+02]
 [1.2490e+03 8.7000e+01 1.6000e+01 1.2000e+01 3.6390e+03 5.0000e+00
  7.0000e+00 9.0000e+00 7.5100e+02 6.7000e+01 6.0000e+00 1.4900e+03
  3.1000e+01 8.0000e+00 1.4130e+03 8.0000e+00 5.0000e+00 7.8500e+02
  2.0400e+02 6.9000e+01 1.3800e+03 3.6000e+01 8.0000e+00 5.0000e+00
  1.0450e+03 5.0000e+00]
 [3.1220e+03 6.0000e+00 3.9800e+02 3.1000e+01 3.6740e+03 6.0000e+00
  8.0000e+00 3.7850e+03 1.5790e+03 5.0000e+00 1.0210e+03 8.8200e+02
  7.0000e+00 1.5000e+01 5.1460e+03 6.0000e+00 2.7000e+01 9.0500e+02
  1.2200e+02 2.2050e+03 6.8200e+02 5.0000e+00 8.0000e+00 5.0000e+00
  2.1600e+02 6.0000e+00]
 [1.4230e+03 2.6000e+01 3.8000e+01 2.8500e+02 4.6730e+03 1.9000e+01
  2.8300e+02 1.5000e+01 2.4720e+03 3.1000

In [12]:
## re-estimate 
def reestimate(alpha,beta,gammas,de_gammas,obv,N,M,T):
#     print("reestimate")
    ##for pi
    for i in range(0, N):
        pi[i] = gammas[0,i]
    ##for A
#     a = np.zeros((N,N))
#     for i in range(0, N):
#         for j in range(0,N):
#             numer = 0
#             denom = 0
#             for t in range(0, T-1):
#                 numer += de_gammas[t,i,j]
#                 denom += gammas[t,i]
#             a[i,j] = numer/denom
    ##for B
    b = np.zeros((N,M))
    for i in range(0,N):
        for j in range(0, M):
            numer = 0
            denom = 0
            for t in range(0, T):
                if obv[t] == j:
                    numer += gammas[t,i]
                denom += gammas[t,i]
            b[i,j] = numer/denom
#     return a,b,pi
    return b,pi
# a,b,pi = reestimate(alpha=alpha,beta=beta,gammas=gammas,de_gammas=de_gammas,obv=obv,N=N,M=M,T=T)
b,pi = reestimate(alpha=alpha,beta=beta,gammas=gammas,de_gammas=de_gammas,obv=obv,N=N,M=M,T=T)


In [13]:
print(a)
print("b")
print(b)
print("pi")
print(pi)
print("alpha")
print(alpha)
print("beta")
print(beta)
print("gammas")
print(gammas)
print("de_gammas")
print(de_gammas)

[[1.1000e+01 1.1400e+03 2.5170e+03 2.4940e+03 5.1000e+01 5.3600e+02
  1.3440e+03 1.0000e+02 2.2330e+03 7.2000e+01 6.8600e+02 6.2100e+03
  1.9620e+03 1.1997e+04 9.1000e+01 9.9200e+02 9.0000e+00 7.1660e+03
  5.8320e+03 8.7290e+03 7.9200e+02 1.1980e+03 4.0600e+02 2.0000e+02
  2.0690e+03 1.1400e+02]
 [1.2490e+03 8.7000e+01 1.6000e+01 1.2000e+01 3.6390e+03 5.0000e+00
  7.0000e+00 9.0000e+00 7.5100e+02 6.7000e+01 6.0000e+00 1.4900e+03
  3.1000e+01 8.0000e+00 1.4130e+03 8.0000e+00 5.0000e+00 7.8500e+02
  2.0400e+02 6.9000e+01 1.3800e+03 3.6000e+01 8.0000e+00 5.0000e+00
  1.0450e+03 5.0000e+00]
 [3.1220e+03 6.0000e+00 3.9800e+02 3.1000e+01 3.6740e+03 6.0000e+00
  8.0000e+00 3.7850e+03 1.5790e+03 5.0000e+00 1.0210e+03 8.8200e+02
  7.0000e+00 1.5000e+01 5.1460e+03 6.0000e+00 2.7000e+01 9.0500e+02
  1.2200e+02 2.2050e+03 6.8200e+02 5.0000e+00 8.0000e+00 5.0000e+00
  2.1600e+02 6.0000e+00]
 [1.4230e+03 2.6000e+01 3.8000e+01 2.8500e+02 4.6730e+03 1.9000e+01
  2.8300e+02 1.5000e+01 2.4720e+03 3.1000

In [14]:
logProb = 0
for i in range(0, T):
    logProb += math.log(c[i])
logProb = -logProb
maxIters = 100
iters = 0
oldLogProb = -math.inf
while iters < maxIters and logProb > oldLogProb:
    iters += 1
    oldLogProb = logProb
    alpha = alpha_t(a=a,b=b,c=c,obv=obv,T=T,N=N)
    beta = backward(a=a,b=b,c=c,T=T,N=N)
    gammas, de_gammas = cal_gammas(a=a,b=b,alpha=alpha,beta=beta,T=T,N=N)
#     a,b,pi = reestimate(alpha=alpha,beta=beta,gammas=gammas,de_gammas=de_gammas,obv=obv,N=N,M=M,T=T)
    b, pi = reestimate(alpha=alpha,beta=beta,gammas=gammas,de_gammas=de_gammas,obv=obv,N=N,M=M,T=T)
    for i in range(0, T):
        logProb += math.log(c[i])
    print(iters)
    print(a)
    print("b")
    print(b)
    print("pi")
    print(pi)
    

1
[[1.1000e+01 1.1400e+03 2.5170e+03 2.4940e+03 5.1000e+01 5.3600e+02
  1.3440e+03 1.0000e+02 2.2330e+03 7.2000e+01 6.8600e+02 6.2100e+03
  1.9620e+03 1.1997e+04 9.1000e+01 9.9200e+02 9.0000e+00 7.1660e+03
  5.8320e+03 8.7290e+03 7.9200e+02 1.1980e+03 4.0600e+02 2.0000e+02
  2.0690e+03 1.1400e+02]
 [1.2490e+03 8.7000e+01 1.6000e+01 1.2000e+01 3.6390e+03 5.0000e+00
  7.0000e+00 9.0000e+00 7.5100e+02 6.7000e+01 6.0000e+00 1.4900e+03
  3.1000e+01 8.0000e+00 1.4130e+03 8.0000e+00 5.0000e+00 7.8500e+02
  2.0400e+02 6.9000e+01 1.3800e+03 3.6000e+01 8.0000e+00 5.0000e+00
  1.0450e+03 5.0000e+00]
 [3.1220e+03 6.0000e+00 3.9800e+02 3.1000e+01 3.6740e+03 6.0000e+00
  8.0000e+00 3.7850e+03 1.5790e+03 5.0000e+00 1.0210e+03 8.8200e+02
  7.0000e+00 1.5000e+01 5.1460e+03 6.0000e+00 2.7000e+01 9.0500e+02
  1.2200e+02 2.2050e+03 6.8200e+02 5.0000e+00 8.0000e+00 5.0000e+00
  2.1600e+02 6.0000e+00]
 [1.4230e+03 2.6000e+01 3.8000e+01 2.8500e+02 4.6730e+03 1.9000e+01
  2.8300e+02 1.5000e+01 2.4720e+03 3.10

In [15]:
print("a")
print(a)
print("b")
print(b)
print("pi")
print(pi)

a
[[1.1000e+01 1.1400e+03 2.5170e+03 2.4940e+03 5.1000e+01 5.3600e+02
  1.3440e+03 1.0000e+02 2.2330e+03 7.2000e+01 6.8600e+02 6.2100e+03
  1.9620e+03 1.1997e+04 9.1000e+01 9.9200e+02 9.0000e+00 7.1660e+03
  5.8320e+03 8.7290e+03 7.9200e+02 1.1980e+03 4.0600e+02 2.0000e+02
  2.0690e+03 1.1400e+02]
 [1.2490e+03 8.7000e+01 1.6000e+01 1.2000e+01 3.6390e+03 5.0000e+00
  7.0000e+00 9.0000e+00 7.5100e+02 6.7000e+01 6.0000e+00 1.4900e+03
  3.1000e+01 8.0000e+00 1.4130e+03 8.0000e+00 5.0000e+00 7.8500e+02
  2.0400e+02 6.9000e+01 1.3800e+03 3.6000e+01 8.0000e+00 5.0000e+00
  1.0450e+03 5.0000e+00]
 [3.1220e+03 6.0000e+00 3.9800e+02 3.1000e+01 3.6740e+03 6.0000e+00
  8.0000e+00 3.7850e+03 1.5790e+03 5.0000e+00 1.0210e+03 8.8200e+02
  7.0000e+00 1.5000e+01 5.1460e+03 6.0000e+00 2.7000e+01 9.0500e+02
  1.2200e+02 2.2050e+03 6.8200e+02 5.0000e+00 8.0000e+00 5.0000e+00
  2.1600e+02 6.0000e+00]
 [1.4230e+03 2.6000e+01 3.8000e+01 2.8500e+02 4.6730e+03 1.9000e+01
  2.8300e+02 1.5000e+01 2.4720e+03 3.10

In [16]:
obv = []
ori = []
key = 1
with open("/Users/chun/IdeaProjects/CS-271/brown.txt", 'r') as file:
    for line in file:
        for char in line:
            if char.isalpha() and 'a' <= char.lower() <= 'z':
                char = char.lower()
                #print(char.lower())
                ori.append(char.lower())
                #shifted.append(chr(((ord(char) - ord('a') + key) % 26) + ord('a')))
                obv.append((ord(char.lower()) - ord('a') + key )%26)
            else:
                continue